# Task 3: Implement a Support Vector Machine (SVM) to classify images of cats and dogs

## Cat vs Dog Image Classifier - Training Script

This script is responsible for training a **Support Vector Machine (SVM)** model to classify images of cats and dogs.

### It performs the following steps:

1. **Loads and preprocesses images** from the training folder.
2. **Extracts HOG (Histogram of Oriented Gradients) features** from each image.
3. **Reduces feature dimensionality** using PCA (Principal Component Analysis).
4. **Scales the features** for standardization.
5. **Trains an SVM classifier** with hyperparameter tuning using GridSearchCV.
6. **Evaluates the trained model** on a validation set.
7. **Saves the trained model** along with the scaler and PCA transformer for future use.

**Output:** A trained model file named **`svm_model_hog_pca.pkl`** is saved.


## 1. Importing Necessary Libraries

In [1]:
import os                           # For interacting with the file system
import numpy as np                  # For array operations and numerical computations
import pickle                       # To save and load the trained model
from pathlib import Path            # For handling file paths in an OS-independent way

from skimage.io import imread                   # To read image files
from skimage.transform import resize            # To resize images to a fixed size
from skimage.feature import hog                 # To extract HOG features
from skimage.color import rgb2gray              # To convert images to grayscale

from sklearn.model_selection import train_test_split, GridSearchCV  # For splitting data and hyperparameter tuning
from sklearn.preprocessing import StandardScaler                    # For feature normalization
from sklearn.decomposition import PCA                               # To reduce dimensionality of features
from sklearn import svm                                              # Support Vector Machine classifier
from sklearn.metrics import accuracy_score, classification_report    # Evaluation metrics

## 2. Setting the Base Directory

In [2]:
train_img_dir = "train/train"  # Folder should contain labeled images

## 3. Defining Image Preprocessing & Feature Extraction Function

In [3]:
# Function to load images and extract either raw pixel or HOG features
# I wrote this function to load images, resize them to a fixed shape, convert them to grayscale
# and then either flatten the raw pixels or extract HOG features depending on the `feature_type`.
# HOG captures edge/texture info which helps distinguish between cats and dogs.

def load_training_data(image_dir, img_size=(128, 128), feature_type='pixels'):
    input_images = []   # To store image features
    output_labels = []  # To store corresponding labels (0 for cat, 1 for dog)

    for file in os.listdir(image_dir):
        if file.endswith(".jpg") or file.endswith(".png"):
            file_path = os.path.join(image_dir, file)
            img = imread(file_path)   # Read image from disk
            
            # Remove alpha channel if image has 4 channels
            if img.shape[-1] == 4:
                img = img[:, :, :3]

            # Resize image and preserve its range
            img_gray = resize(img, img_size, anti_aliasing=True, preserve_range=True)

            # Convert RGB image to grayscale (required for HOG)
            if img_gray.ndim == 3 and img_gray.shape[2] == 3:
                img_gray = rgb2gray(img_gray)

            # Feature extraction: either use raw pixels or HOG features
            if feature_type == 'pixels':
                input_images.append(img_gray.flatten())
            elif feature_type == 'hog':
                fd = hog(img_gray, orientations=9, pixels_per_cell=(8, 8),
                         cells_per_block=(2, 2), visualize=False, channel_axis=None)
                input_images.append(fd)

            # Label assignment: 'cat' → 0, 'dog' → 1
            label = 0 if 'cat' in file.lower() else 1
            output_labels.append(label)

    return np.array(input_images), np.array(output_labels)

## 4. Training SVM with Grid Search

In [4]:
# This function trains an SVM model using GridSearch to find optimal hyperparameters
# I split the data into train and test sets, scale it, perform grid search on C and gamma
# and evaluate performance using accuracy and classification report

def train_svm_model_with_gridsearch(X, y):
    # Split into training and validation sets (75%-25% split)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

    # Standardize the features to have zero mean and unit variance
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 0.1, 0.01],
        'kernel': ['rbf']
    }

    # Initialize SVM with probability output enabled
    svc = svm.SVC(probability=True)

    # Use GridSearchCV to find the best combination of hyperparameters
    clf = GridSearchCV(svc, param_grid, cv=5, n_jobs=-1, verbose=2)
    print("Starting Grid Search...")
    clf.fit(x_train_scaled, y_train)

    print(f"Best Parameters: {clf.best_params_}")

    # Evaluate performance on validation set
    y_pred = clf.predict(x_test_scaled)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(classification_report(y_test, y_pred, target_names=['Cat', 'Dog']))

    return clf.best_estimator_, scaler  # Return best model and the scaler used

## 5. Loading Data and Applying HOG

In [5]:
# I use HOG because it captures edge and texture details that help differentiate cats from dogs
print("Loading data and extracting HOG features...")
X_data, y_data = load_training_data(train_img_dir, img_size=(128, 128), feature_type='hog')

Loading data and extracting HOG features...


## 6. Reducing Dataset Size 

In [6]:
# Reduce data to 8000 samples for faster training
X_data, _, y_data, _ = train_test_split(
    X_data, y_data,
    train_size=8000,
    stratify=y_data,
    random_state=42
)
print(f"Original feature dimension after HOG: {X_data.shape[1]}")

Original feature dimension after HOG: 8100


## 7. Applying PCA for Dimensionality Reduction

In [7]:
# I apply PCA to reduce noise and computation time; 500 components preserve most variance
pca = PCA(n_components=500)
X_data_reduced = pca.fit_transform(X_data)

print(f"Reduced feature dimension after PCA: {X_data_reduced.shape[1]}")
print(f"Explained variance ratio by 500 components: {np.sum(pca.explained_variance_ratio_):.2f}")

Reduced feature dimension after PCA: 500
Explained variance ratio by 500 components: 0.67


## 8. Training the Final Model

In [8]:
# Train the model using the processed features
model, scaler = train_svm_model_with_gridsearch(X_data_reduced, y_data)

Starting Grid Search...
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Accuracy: 71.35%
              precision    recall  f1-score   support

         Cat       0.71      0.73      0.72      1000
         Dog       0.72      0.70      0.71      1000

    accuracy                           0.71      2000
   macro avg       0.71      0.71      0.71      2000
weighted avg       0.71      0.71      0.71      2000



## 9. Saving the Model

In [9]:
# Save the trained model, scaler, and PCA transformer to a pickle file for reuse
model_path = "svm_model_hog_pca.pkl"
with open(model_path, 'wb') as file:
    pickle.dump((model, scaler, pca), file)

print(f"Model saved at: {model_path}")

Model saved at: svm_model_hog_pca.pkl
